In [1078]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import spacy
from spacy.tokens import Span
from spacy.pipeline import EntityRuler
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.matcher import PhraseMatcher
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
#from nltk import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
import string
import numpy as np

URL = 'https://www.greencarreports.com/news/first-drives'
page = requests.get(URL)

soup = BeautifulSoup(page.content,'html.parser')

#nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

#sp = spacy.load('en_core_web_sm')
sp = spacy.load('en_core_web_lg')

# Allow print to format None values
def xstr(s):
    if s is None:
        return ''
    else:
        return str(s)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ping\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ping\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Ping\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Ping\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [542]:
# Get list of reviews from the homepage

links = soup.find_all(class_='img-wrap',
                    href=True)

for a in links:
    print('https://www.greencarreports.com' + a['href'])
   

https://www.greencarreports.com/news/1134484_first-drive-review-2022-hyundai-ioniq-5-plugs-into-new-wave-not-the-mainstream
https://www.greencarreports.com/news/1133945_first-drive-review-2022-volvo-xc60-recharge-plug-in-hybrid-delivers-more-range-and-power
https://www.greencarreports.com/news/1133898_first-drive-review-2022-volvo-c40-recharge-trades-function-for-style
https://www.greencarreports.com/news/1133831_first-drive-review-2021-ford-mustang-mach-e-gt-upgrades-performance-without-losing-range
https://www.greencarreports.com/news/1133777_first-drive-review-2022-lucid-air-delivers-a-new-leading-edge-for-evs
https://www.greencarreports.com/news/1133704_review-rivian-r1t-photos-price-review-specs-info
https://www.greencarreports.com/news/1133504_first-drive-review-2022-polestar-2-dual-motor-revisits-the-value-equation
https://www.greencarreports.com/news/1133505_first-drive-review-2022-polestar-2-single-motor-beats-tesla-on-build-quality-remixes-priorities
https://www.greencarrepor

In [754]:
# Build and test for particular reviews

#URL2 = 'https://www.greencarreports.com/news/1133945_first-drive-review-2022-volvo-xc60-recharge-plug-in-hybrid-delivers-more-range-and-power'
#URL2 = 'https://www.greencarreports.com/news/1133898_first-drive-review-2022-volvo-c40-recharge-trades-function-for-style'
#URL2 = 'https://www.greencarreports.com/news/1133777_first-drive-review-2022-lucid-air-delivers-a-new-leading-edge-for-evs'
#URL2 = 'https://www.greencarreports.com/news/1133104_first-drive-review-2021-hyundai-santa-fe-hybrid-finds-a-new-middle-ground-for-efficient-family-suvs'
#URL2 = 'https://www.greencarreports.com/news/1133898_first-drive-review-2022-volvo-c40-recharge-trades-function-for-style'
#URL2 = 'https://www.greencarreports.com/news/1134468_gmc-sierra-electric-pickup-due-in-2023-with-denali-luxury-hummer-ev-design-cues'
URL2 = 'https://www.greencarreports.com/news/1130545_first-drive-2021-volkswagen-id-4-is-buzzworthy-but-a-software-update-away-from-breakthrough'
page2 = requests.get(review URL2)

soup2 = BeautifulSoup(page2.content,'html.parser')

results2 = soup2.find(class_='article-title')
#results2 = soup2.find('p', {'dir' : 'ltr'})     # Only works for some of the review pages
title = results2.get_text()
print(title)

year = re.findall("20\d\d", title)
year = int(year[0])
x = re.search(str(year), title)
x.span()
results2




First drive review: 2021 Volkswagen ID.4 is buzzworthy, but a software update away from breakthrough


<h1 class="article-title"><span>First drive review: 2021 Volkswagen ID.4 is buzzworthy, but a software update away from breakthrough</span></h1>

In [947]:
soup2 = BeautifulSoup(page2.content,'html.parser')
res2 = soup2.find(class_='article-body').find_all('p')
#res2 = ''.join(res2) 
#print(res2)

text = ''
for b in res2:
    if b.parent.name != 'div':       # Gets rid of nested figure captions 
        text = text + ' ' + b.get_text(separator=' ')

text

" The 2021 Volkswagen ID.4 electric SUV is likely to turn many more Americans on to the virtues of EVs.\xa0 Based on all the bugs and hiccups we've seen in its interface, it's not perfect. But a short, one-hour drive among some of the San Francisco Bay Area’s hilly backroads and a stretch on the highway, was enough to confirm some first suspicions and impressions regarding the ID.4, and we came away more confident about its mission. Read this rundown of  2021 VW ID.4 hits and misses  for more up-close takeaways on the ID.4, after we've spent a little more time with this important EV, or read on for those first impressions. It’s not quick, but that’s okay The single-motor versions of the ID.4 that will arrive first make 201 horsepower and 228 pound-feet of torque, with that power delivered by a single electric motor at the rear wheels. A dual-motor, all-wheel drive version with 312 hp is slated for later and that one is sure to be the much quicker of the two.\xa0  With a roughly estimat

In [941]:
#soup2 = BeautifulSoup(page2.content,'html.parser')
#res2 = soup2.find(class_='article-body')
        
#for d in res2.descendants:
#    if d.name == 'p' and d.get('class', '') == ['image_wrapper']:
#        print(d.text)


In [547]:
sen = sp(title)
named_ents = sen.ents
car_make = None
car_model = None

for entity in named_ents:
    print(entity.text + ' - ' + entity.label_)
    if entity.label_== 'ORG':
        car_make = entity.text
    elif entity.label_ == 'PRODUCT':
        car_model = entity.text

print(year, car_make, car_model)


First - ORDINAL
2022 - CARDINAL
Volvo - ORG
2022 Volvo 


In [548]:
sen = sp(title)
named_ents = sen.ents

make = None
model = None

for entity in named_ents:
    print(entity.text + ' - ' + entity.label_)
    # The first entity found is usually correct
    if (entity.label_== 'ORG') and (make==None):
        make = entity.text
    elif entity.label_ == 'PRODUCT'and (model==None):
        model = entity.text

print(year, make, model)

First - ORDINAL
2022 - CARDINAL
Volvo - ORG
2022 Volvo None


In [549]:
# Compare using NLTK library
tokens = nltk.word_tokenize(title)
tagged = nltk.pos_tag(tokens)
named_ents_nltk = nltk.ne_chunk(tagged)
print(named_ents_nltk)

(S
  (GPE First/NNP)
  drive/NN
  review/NN
  :/:
  2022/CD
  (PERSON Volvo/NNP C40/NNP Recharge/NNP)
  trades/VBZ
  function/NN
  for/IN
  style/NN)


In [550]:
tagged = nltk.pos_tag(tokens)
print(tagged)
for t in tagged:
    if t[1]=='VBZ':     # Verb to match singular third person
        print(t[0])
        

[('First', 'NNP'), ('drive', 'NN'), ('review', 'NN'), (':', ':'), ('2022', 'CD'), ('Volvo', 'NNP'), ('C40', 'NNP'), ('Recharge', 'NNP'), ('trades', 'VBZ'), ('function', 'NN'), ('for', 'IN'), ('style', 'NN')]
trades


In [551]:
sen = sp(title)

for i in range(1,len(sen)):
    if sen[i].tag_=='VBZ':
        print(sen[i])

In [552]:
tokens = nltk.word_tokenize('The 2022 Mercedes-Benz EQS pitches the luxury nameplate')
tokens

['The',
 '2022',
 'Mercedes-Benz',
 'EQS',
 'pitches',
 'the',
 'luxury',
 'nameplate']

In [553]:
# Do it for all product reviews
rev_list = []
keys_ = []
for a in links:
    rev_list.append('https://www.greencarreports.com' + a['href'])
    page_temp = requests.get(rev_list[-1])
    soup_temp = BeautifulSoup(page_temp.content,'html.parser')
    
    res = soup_temp.find(class_='article-title')
    title = res.get_text()
    year = re.findall("20\d\d", title)
    year = int(year[0])
    x = re.search(str(year), title)
    rest = title[(x.span()[1]+1):]
    
    tokens = nltk.word_tokenize(rest)
    print(tokens[0])
   

Hyundai
Volvo
Volvo
Ford
Lucid
Rivian
Polestar
Polestar
Hyundai
Audi
Mercedes-Benz
BMW
Hyundai
Karma
Chevrolet
Volkswagen
Kia
Toyota
Ford
Volvo
Hyundai
Honda
Toyota


IndexError: list index out of range

In [554]:
 # Do it for all product reviews
rev_list = []
keys_ = []

for a in links:
    rev_list.append('https://www.greencarreports.com' + a['href'])
    page_temp = requests.get(rev_list[-1])
    soup_temp = BeautifulSoup(page_temp.content,'html.parser')
    
    res = soup_temp.find(class_='article-title')
    title = res.get_text()
    year = re.findall("20\d\d", title)
    if len(year) == 1:
        year = int(year[0])
        x = re.search(str(year), title)
        rest = title[(x.span()[1]+1):]
    else:
        # Atypical title format, skip
        continue 
    
    tokens = nltk.word_tokenize(rest)
    car_make = tokens[0]
    x2 = re.search(car_make, title)
    
    sen = sp(rest)
    named_ents = sen.ents
   
    # Leaving the car_make in for entity detection in case it contextually helps identify the car model
    # Next, remove car make from the entity to get car model
    car_model = None
    for entity in named_ents:
        if (entity.label_== 'ORG') or (entity.label_ == 'PRODUCT'):
            if (entity.text == car_make):
                car_make_checked = entity.text            
            elif (car_model==None):
                str1 = entity.text
                car_model = str1.replace(car_make,'')
        
    keys_.append([year, car_make, car_model])
    print(year,'|', car_make, '|', car_model)


2022 | Hyundai | None
2022 | Volvo |  XC60 Recharge
2022 | Volvo | None
2021 | Ford |  Mustang Mach-E GT
2022 | Lucid | EVs
2022 | Rivian | None
2022 | Polestar | None
2022 | Polestar | None
2021 | Hyundai |  Santa Fe Hybrid
2022 | Audi |  E-Tron GT
2022 | Mercedes-Benz |  EQS
2021 | BMW | 330e
2022 | Hyundai | Tucson Hybrid
2021 | Karma | None
2022 | Chevrolet |  Bolt EUV
2021 | Volkswagen |  ID.4
2021 | Kia | Sorento Hybrid
2021 | Toyota | None
2021 | Ford |  Mustang Mach-E electric SUV
2021 | Volvo |  XC40 Recharge electric SUV
2021 | Hyundai | Elantra Hybrid
2021 | Honda | Accord Hybrid
2021 | Toyota | Sienna
2022 | Hyundai | None
2022 | finalist | None
2023 | with | Denali


IndexError: list index out of range

In [832]:
# Do it for all product reviews with guesses for car model
rev_list = []
keys = []
car_makes_all = []
car_makes_known = []
car_models_all = []
car_name_words = []

print(f"{'Year':{10}} {'Make':{15}} {'Model':{15}}")

for a in links:
    page_temp = requests.get('https://www.greencarreports.com' + a['href'])
    soup_temp = BeautifulSoup(page_temp.content,'html.parser')
    
    res = soup_temp.find(class_='article-title')
    title = res.get_text()
    year = re.findall("20\d\d", title)
    if (len(year)==1):
        year = int(year[0])
        x = re.search(str(year), title)
        rest = title[(x.span()[1]+1):]
        if len(rest)==0:
            # Atypical title format, skip
            continue
    else:
        # Atypical title format, skip
        continue 
    
    tokens = nltk.word_tokenize(rest)
    car_make = tokens[0]
    x2 = re.search(car_make, title)
    
    sen = sp(rest)
    named_ents = sen.ents

    # Leaving the car_make in for entity detection in case it contextually helps identify the car model
    # Next, remove car make from the entity to get car model
    car_model = None
    for entity in named_ents:
        if (entity.label_== 'ORG') or (entity.label_ == 'PRODUCT'):
            if (entity.text == car_make):
                if entity.label_ == 'ORG':
                    car_makes_known = car_makes_known + [entity.text] 
            elif (car_model==None):
                str1 = entity.text
                car_model = str1.replace(car_make,'')

    if car_model==None:
        VBZ_word = None
      
        for i in range(1,len(sen)):
            if (sen[i].tag_=='VBZ') and (VBZ_word==None):
                VBZ_word = sen[i].text
               
        if VBZ_word==None:      # Spacy tagger couldn't find it, try NLTK
            tagged = nltk.pos_tag(tokens)
            for t in tagged:
                if (t[1]=='VBZ') and (VBZ_word==None):     # First instance of Verb to match singular third person
                    VBZ_word = t[0]
        
        if VBZ_word==None:
            continue
        else:
            inds1 = title.find(car_make)
            inds2 = title.find(VBZ_word)
            car_model = title[inds1+len(car_make)+1:inds2-1]
    
    car_make = car_make.strip()
    if not(car_make[0].isupper()):    # Make is not capitalized - likely wrong
        car_make = None
    else:
        car_makes_all = car_makes_all + [car_make]
        car_name_words = car_name_words + [car_make.lower()]
        
    car_model = car_model.strip()
    if not(car_model[0].isupper() or car_model[0].isdigit()):  
        car_model = None
    else:
        car_models_all = car_models_all + [car_model]
        car_name_words = car_name_words + car_model.lower().split()
                
    keys_.append([year, car_make, car_model])
    rev_list.append('https://www.greencarreports.com' + a['href'])

    print(f'{xstr(year):{10}} {xstr(car_make):{15}} {xstr(car_model):{15}}')
    
# Get unique entries
car_makes_all = set(car_makes_all)
car_makes_known = set(car_makes_known)
car_makes_unknown = car_makes_all - car_makes_known
car_models_all = set(car_models_all)
car_name_words = set(car_name_words)

print(car_makes_all)
print(car_makes_known)
print(car_makes_unknown)


Year       Make            Model          
2022       Volvo           XC60 Recharge  
2022       Volvo           C40 Recharge   
2021       Ford            Mustang Mach-E GT
2022       Lucid           EVs            
2022       Rivian          R1T            
2022       Polestar        2 dual-motor   
2022       Polestar        2 single motor 
2021       Hyundai         Santa Fe Hybrid
2022       Audi            E-Tron GT      
2022       Mercedes-Benz   EQS            
2021       BMW             330e           
2022       Hyundai         Tucson Hybrid  
2021       Karma           GS-6           
2022       Chevrolet       Bolt EUV       
2021       Volkswagen      ID.4           
2021       Kia             Sorento Hybrid 
2021       Toyota          Mirai          
2021       Ford            Mustang Mach-E electric SUV
2021       Volvo           XC40 Recharge electric SUV
2021       Hyundai         Elantra Hybrid 
2021       Honda           Accord Hybrid  
2021       Toyota          Si

In [482]:
# Correct individual errors
keys_[3][2] = 'Air'
keys_[-1][1] = 'GMC'
keys_


[[2022, 'Volvo', 'XC60 Recharge'],
 [2022, 'Volvo', 'C40 Recharge'],
 [2021, 'Ford', 'Mustang Mach-E GT'],
 [2022, 'Lucid', 'Air'],
 [2022, 'Rivian', 'R1T'],
 [2022, 'Polestar', '2 dual-motor'],
 [2022, 'Polestar', '2 single motor'],
 [2021, 'Hyundai', 'Santa Fe Hybrid'],
 [2022, 'Audi', 'E-Tron GT'],
 [2022, 'Mercedes-Benz', 'EQS'],
 [2021, 'BMW', '330e'],
 [2022, 'Hyundai', 'Tucson Hybrid'],
 [2021, 'Karma', 'GS-6'],
 [2022, 'Chevrolet', 'Bolt EUV'],
 [2021, 'Volkswagen', 'ID.4'],
 [2021, 'Kia', 'Sorento Hybrid'],
 [2021, 'Toyota', 'Mirai'],
 [2021, 'Ford', 'Mustang Mach-E electric SUV'],
 [2021, 'Volvo', 'XC40 Recharge electric SUV'],
 [2021, 'Hyundai', 'Elantra Hybrid'],
 [2021, 'Honda', 'Accord Hybrid'],
 [2021, 'Toyota', 'Sienna'],
 [2021, 'Toyota', 'Venza'],
 [2023, None, 'Denali'],
 [2022, 'Volvo', 'XC60 Recharge'],
 [2022, 'Volvo', 'C40 Recharge'],
 [2021, 'Ford', 'Mustang Mach-E GT'],
 [2022, 'Lucid', 'EVs'],
 [2022, 'Rivian', 'R1T'],
 [2022, 'Polestar', '2 dual-motor'],
 [20

In [838]:
car_name_words = set.union(car_name_words,{'tesla','s','y'})    # add tesla models to excluded words
car_name_words

{'2',
 '330e',
 'accord',
 'audi',
 'bmw',
 'bolt',
 'c40',
 'chevrolet',
 'denali',
 'dual-motor',
 'e-tron',
 'elantra',
 'electric',
 'eqs',
 'euv',
 'evs',
 'fe',
 'ford',
 'gs-6',
 'gt',
 'honda',
 'hybrid',
 'hyundai',
 'id.4',
 'karma',
 'kia',
 'lucid',
 'mach-e',
 'mercedes-benz',
 'mirai',
 'motor',
 'mustang',
 'polestar',
 'r1t',
 'recharge',
 'rivian',
 's',
 'santa',
 'sienna',
 'single',
 'sorento',
 'suv',
 'tesla',
 'toyota',
 'tucson',
 'volkswagen',
 'volvo',
 'xc40',
 'xc60',
 'y'}

In [557]:
car_makes_unknown.discard(None)

car_makes_unknown2 = {x for x in car_makes_unknown if not((len(sp(x).ents)>0) and (sp(x).ents[0].label_=='ORG'))}

print("Suggested car makes to add to NER:")
print(car_makes_unknown2)

Suggested car makes to add to NER:
{'Rivian', 'Polestar', 'Lucid', 'Karma'}


Human should approve suggested additions as names of car manufacturing companies. Rather than adding named entities by training the large English model, which sounds slow and tricky, I chose to use rule-based entity recognition.

In [558]:
ruler = sp.add_pipe("entity_ruler")
for cm in list(car_makes_unknown2):
    ruler.add_patterns([{"label": 'ORG', "pattern": cm}])

# Test new labels
doc = sp("New car models from Rivian")
for ent in doc.ents:
    print(ent.text , '->', ent.label_)
    

Rivian -> ORG


Decided it doesn't make sense to add model names without the make as they are often too generic, e.g., '2' or 'Denali'

In [1053]:
# Find and combine text from the body of the review
text_all = ''
text_dict = {}

for i in range(0,len(rev_list)):
    page_temp = requests.get(rev_list[i])
    soup_temp = BeautifulSoup(page_temp.content,'html.parser')
    res = soup_temp.find(class_='article-body').find_all('p')
    text = ''
    for b in res:
        if b.parent.name != 'div':       # Gets rid of nested figure captions 
            text = text + '\n' + b.get_text(separator=' ')
    
    text_all = text_all + '\n' + text
    text_dict[tuple(keys_[i])] = text

In [1051]:
# Top nouns
doc = sp(text_all)
doc = [i.text.lower() for i in doc if i.text not in STOP_WORDS and i.text not in string.punctuation and i.tag_=="NN"]
# Or stopwords.words('english')
doc=pd.Series(doc)
top10_NN=doc.value_counts()[:20].index.tolist() 
top10_NN

['drive',
 'range',
 'system',
 'car',
 'wheel',
 'power',
 'motor',
 'mode',
 'battery',
 'mph',
 'inch',
 'vehicle',
 'time',
 'way',
 'road',
 'pedal',
 'speed',
 'steering',
 'highway',
 'driving']

In [951]:
doc.value_counts()

range         109
drive         108
system        104
car            98
wheel          80
             ... 
philosophy      1
bonus           1
snappiness      1
paper           1
plant           1
Length: 1374, dtype: int64

In [566]:
sentences = text_all.split(' ')

tokens = nltk.word_tokenize(text_all)
phrases = Phrases(tokens,
                  min_count=2,
                  threshold=5.0)
phr = Phraser(phrases)
print(phr)

FrozenPhrases<0 phrases, min_count=2, threshold=5.0>


I'm reading the text database has to be a large corpus to work with Phraser.

In [969]:
bigram = list(ngrams(text_all.lower().split(),2))
bigram = (b for b in bigram if b[0] not in STOP_WORDS and b[1] not in STOP_WORDS and b[0] not in car_name_words and b[1] not in car_name_words and b[0] not in string.punctuation and b[1] not in string.punctuation)
bigram = pd.Series(bigram)
top_2g = [w for w in bigram.value_counts().index if bigram.value_counts()[w]>5] 
top_2g = (' '.join(w) for w in top_2g)

top_2g = list(top_2g)
top_2g = pd.Series(top_2g)
keep = np.ones(len(top_2g),dtype=bool)

for i in range(1,len(top_2g)):
    #top_2g[i] = re.sub(r'[^\w\s]', '', top_2g[i])   # To agressive, you want to keep some apostrophes and hyphens
    top_2g[i] = top_2g[i].replace(',','')
    doc = sp(top_2g[i])    
    for ent in doc.ents:
        if ent.label_=='GPE':
            keep[i] = False

top_2g = list(top_2g[keep].values)

# Filtering out geographical locations removes much of the list, e.g., "Los Angeles"

top_2g


['0-60 mph',
 'regenerative braking',
 'gas engine',
 'apple carplay',
 'battery pack',
 'trip computer',
 'one-pedal driving',
 'performance edition',
 'car reports',
 'green car',
 'doesn’t feel',
 'all-wheel drive',
 'mph time',
 'rear seat',
 'android auto',
 'fuel cell',
 'mpg combined',
 'feel like',
 'epa estimated',
 'brake pedal',
 'federal tax',
 'wireless smartphone',
 'heat pump',
 'mpg city',
 'drive modes',
 'steering wheel',
 '60 mph']

In [851]:
doc = sp('december 2020')    
ent.label_    # Shouldn't this be date?

'QUANTITY'

In [968]:
trigram = list(ngrams(text_all.lower().split(),3))
trigram = (b for b in trigram if b[0] not in STOP_WORDS and 
           b[1] not in STOP_WORDS and 
           b[2] not in STOP_WORDS and
           b[0] not in car_name_words and 
           b[1] not in car_name_words and 
           b[2] not in car_name_words and
           b[0] not in string.punctuation and 
           b[1] not in string.punctuation and
           b[2] not in string.punctuation)

trigram = pd.Series(trigram)
top_3g = [w for w in trigram.value_counts().index if trigram.value_counts()[w]>3] 
#top20t = trigram.value_counts()[:20].index.tolist() 
top_3g = (' '.join(w) for w in top_3g)

top_3g = list(top_3g)
top_3g = pd.Series(top_3g)
keep = np.ones(len(top_3g),dtype=bool)

for i in range(1,len(top_3g)):
    top_3g[i] = top_3g[i].replace(',','')
    doc = sp(top_3g[i])    
    for ent in doc.ents:
        if ent.label_=='GPE':
            keep[i] = False

top_3g = list(top_3g[keep].values)

top_3g

['green car reports',
 '0-60 mph time',
 'drive (eqs 580)',
 '12.3-inch digital gauge',
 'wireless apple carplay',
 'total system output']

In [971]:
fourgram = list(ngrams(text_all.lower().split(),4))
fourgram = (b for b in fourgram if b[0] not in STOP_WORDS and 
           b[1] not in STOP_WORDS and 
           b[2] not in STOP_WORDS and
           b[3] not in STOP_WORDS and
           b[0] not in car_name_words and 
           b[1] not in car_name_words and 
           b[2] not in car_name_words and
           b[3] not in car_name_words and        
           b[0] not in string.punctuation and 
           b[1] not in string.punctuation and
           b[2] not in string.punctuation and
           b[3] not in string.punctuation)

fourgram = pd.Series(fourgram)
top_4g = [w for w in fourgram.value_counts().index if fourgram.value_counts()[w]>=3] 
top_4g = (' '.join(w) for w in top_4g)

top_4g = list(top_4g)
top_4g = pd.Series(top_4g)
keep = np.ones(len(top_4g),dtype=bool)

for i in range(1,len(top_4g)):
    top_4g[i] = top_4g[i].replace(',','')
    doc = sp(top_4g[i])    
    for ent in doc.ents:
        if ent.label_=='GPE':
            keep[i] = False

top_4g = list(top_4g[keep].values)

top_4g

['$7,500 federal ev tax']

In [1047]:
# Collate single, bi-, trigram and four-grams NN's

def not_in_list (x, list_against):
    for w in list_against:
        if w.find(x)!=-1:
            return False
    else:
        return True
    
top10_NN_filt = list(filter(lambda x: not_in_list(x, top_2g+top_3g+top_4g), top10_NN))
top_2g_filt = list(filter(lambda x: not_in_list(x, top_3g+top_4g), top_2g))
top_3g_filt = list(filter(lambda x: not_in_list(x, top_4g), top_3g))

print(top_2g_filt)

['regenerative braking', 'gas engine', 'battery pack', 'trip computer', 'one-pedal driving', 'performance edition', 'doesn’t feel', 'all-wheel drive', 'rear seat', 'android auto', 'fuel cell', 'mpg combined', 'feel like', 'epa estimated', 'brake pedal', 'federal tax', 'wireless smartphone', 'heat pump', 'mpg city', 'drive modes', 'steering wheel']


In [1048]:
top_NNs = top10_NN_filt + top_2g_filt + top_3g_filt + top_4g
top_NNs

['range',
 'power',
 'motor',
 'vehicle',
 'way',
 'road',
 'speed',
 'highway',
 'torque',
 'regenerative braking',
 'gas engine',
 'battery pack',
 'trip computer',
 'one-pedal driving',
 'performance edition',
 'doesn’t feel',
 'all-wheel drive',
 'rear seat',
 'android auto',
 'fuel cell',
 'mpg combined',
 'feel like',
 'epa estimated',
 'brake pedal',
 'federal tax',
 'wireless smartphone',
 'heat pump',
 'mpg city',
 'drive modes',
 'steering wheel',
 'green car reports',
 '0-60 mph time',
 'drive (eqs 580)',
 '12.3-inch digital gauge',
 'wireless apple carplay',
 'total system output',
 '$7,500 federal ev tax']

In [703]:
wds = nltk.word_tokenize(text_all)
fdist = nltk.FreqDist([w for w in wds])
bigram_seq = nltk.FreqDist(nltk.bigrams(wds))
bigram_dist = nltk.FreqDist((x.lower(),y.lower())
                                 for (x,y) in bigram_seq)

bigram_dist [('plug-in','hybrid')]   # Only 2 occurrences - cannot use this set of reviews alone to generate probabilities - it's too small a data set

2

In [651]:
# Try word2vec
# Word2vec also needs a training set - the Google News corpus from 2013 onward 
# may be suboptimal for recent and less common car models, 
# but we care about properties not car models here.

doc = text_all.replace("\n", " ")

data = []
  
# iterate through each sentence in the file
for i in sent_tokenize(doc):
    temp = []
      
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
  
    data.append(temp)

# Create CBOW model
#model1 = Word2Vec(data, min_count = 1, 
#                  window = 5)

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

# Print results
print("Cosine similarity between 'plug-in' " + 
               "and 'hybrid' - CBOW : ",
    model.similarity('plug-in', 'hybrid'))
      
print("Cosine similarity between '0-60' " +
                 "and 'time' - CBOW : ",
      model.wv.similarity('0-60', 'time'))

print("Cosine similarity between '0-60' " +
                 "and 'regenerative' - CBOW : ",
      model.wv.similarity('0-60', 'regenerative'))

print("Cosine similarity between 'cow' " +
                 "and 'regenerative' - CBOW : ",
      model.wv.similarity('0-60', 'complex'))
  
model.wv.most_similar('plug-in')

AttributeError: 'KeyedVectors' object has no attribute 'wv'

In [661]:
model.similarity('0-60', 'mph')

# regenerative braking versus straw
# plug-in not there

KeyError: "Key '0-60' not present"

In [643]:
index2word = model1.wv.index_to_key

bigram=list(ngrams(doc.lower().split(),2))
trigram=list(ngrams(doc.lower().split(),3))
print('Bigrams:')
bg = [word for word in index2word for x in bigram if word.lower()==x[0]+'_'+x[1]]


Bigrams:


In [982]:
# Top adjectives

doc = sp(text_all)
doc = [i.text.lower() for i in doc if i.text not in STOP_WORDS and i.text not in string.punctuation and i.tag_=="JJ"]
# Or stopwords.words('english')
doc=pd.Series(doc)
top20_JJ=doc.value_counts()[:20].index.tolist() 
top20_JJ

['electric',
 'rear',
 'new',
 'good',
 'standard',
 'hybrid',
 'long',
 'big',
 'single',
 'high',
 'different',
 'regenerative',
 'dual',
 'low',
 'short',
 'easy',
 'wireless',
 'large',
 'available',
 'small']

In [707]:
# Top comparative adjectives

doc = sp(text_all)
doc = [i.text.lower() for i in doc if i.text not in STOP_WORDS and i.text not in string.punctuation and i.tag_=="JJR"]
# Or stopwords.words('english')
doc=pd.Series(doc)
top10_JJR=doc.value_counts()[:20].index.tolist() 
top10_JJR

['better',
 'larger',
 'lower',
 'smaller',
 'closer',
 'shorter',
 'taller',
 'longer',
 'cheaper',
 'slower',
 'lighter',
 'bigger',
 'quicker',
 'heavier',
 'firmer',
 'sharper',
 'softer',
 'stiffer',
 'higher',
 'smoother']

In [1015]:
pentagram=list(ngrams(text_all.lower().split(),5))
context = [i for i in pentagram if i[2] == 'better']
context

[('does', 'a', 'better', 'job', 'of'),
 ('offer', 'much', 'better', 'lap', 'times'),
 ('which', 'are', 'better', 'than', 'those'),
 ('weight', 'far', 'better', 'than', 'i'),
 ('interior', 'looks', 'better', 'in', 'my'),
 ('model', 'has', 'better', 'ratings', 'of'),
 ('to', 'get', 'better', 'body', 'control'),
 ('glass', 'for', 'better', 'noise', 'and'),
 ('be', 'a', 'better', 'solution', 'for'),
 ('it’s', 'just', 'better', 'than', 'its'),
 ('full', 'stop.', 'better', 'on', 'power,'),
 ('on', 'power,', 'better', 'on', 'efficiency,'),
 ('efficiency,', 'and', 'better', 'even', 'on'),
 ('was', 'noticeably', 'better', 'in', 'the'),
 ('got', 'a', 'better', 'backseat', 'than'),
 ('make', 'a', 'better', 'showing.', 'volkswagen'),
 ('and', 'has', 'better', 'drivability', 'than'),
 ('car', 'is', 'better', 'than', 'the'),
 ('the', 'road', 'better', 'than', 'tesla’s'),
 ('being', 'a', 'better', 'car', 'and'),
 ('still', 'a', 'better', 'experience.', 'versus'),
 ('get', 'a', 'better', 'suspension',

In [698]:
context = [i for i in pentagram if i[2] == 'larger']
context

[('anticipates', 'a', 'larger', 'federal', 'tax'),
 ('2022.', 'the', 'larger', 'battery', 'pack'),
 ('with', 'the', 'larger', 'battery,', 'added'),
 ('motor', 'and', 'larger', 'battery', 'pack,'),
 ('brakes', 'with', 'larger', '19-inch', 'rotors'),
 ('with', 'the', 'larger', '88-kwh', 'battery'),
 ('to', 'fit', 'larger', 'items,', 'or'),
 ('plans', 'a', 'larger', '180-kwh', 'battery'),
 ('the', 'significantly', 'larger', 'toyota', 'highlander'),
 ('though', 'the', 'larger', 'touchscreen', 'ends'),
 ('trim', 'upgrades,', 'larger', 'wheels', 'with'),
 ('with', 'a', 'larger', '10.3-inch', 'screen'),
 ('caveat:', 'the', 'larger', 'screen', 'axes'),
 ('to', 'the', 'larger', 'vehicle,', 'the'),
 ('(with', 'a', 'larger', 'front', 'motor);'),
 ('or', 'a', 'larger', 'duffel', 'bag.'),
 ('from', 'its', 'larger', 'and', 'more'),
 ('add', 'the', 'larger', 'screen', 'and'),
 ('auto', 'feel', 'larger', 'on', 'the'),
 ('had', 'a', 'larger', 'interior', 'than')]

In [699]:
context = [i for i in pentagram if i[2] == 'lower']
context

[('thrust', 'at', 'lower', 'speeds.', 'what’s'),
 ('the', '2.2-inch', 'lower', 'roofline', 'and'),
 ('rides', '0.4-inch', 'lower', 'than', 'the'),
 ('mach-e', 'to', 'lower', 'the', 'center'),
 ('side.', 'the', 'lower', 'screen,', 'called'),
 ('if', 'a', 'lower', 'price', 'point'),
 ('with', 'a', 'lower', 'price', 'point.'),
 ('find', 'at', 'lower', 'speeds.', 'there'),
 ('another', 'touchscreen', 'lower', 'in', 'the'),
 ('at', 'a', 'lower', 'price', 'than'),
 ('understeer', 'at', 'lower', 'speeds', 'and'),
 ('alone', 'at', 'lower', 'speeds.', 'hyundai'),
 ('was', 'at', 'lower', 'speeds,', 'but'),
 ('dash', 'feels', 'lower', 'yet', 'closer'),
 ('much,', 'much', 'lower', 'thirst.', 'toyota'),
 ('selecting', 'a', 'lower', '“gear”', 'increases')]

In [709]:
# Top adverbs
doc = sp(text_all)
doc = [i.text.lower() for i in doc if i.text not in STOP_WORDS and i.text not in string.punctuation and i.tag_=="RB"]
# Or stopwords.words('english')
doc=pd.Series(doc)
top10_RB=doc.value_counts()[:10].index.tolist() 
top10_RB

['far',
 'nearly',
 'instead',
 'actually',
 'fast',
 'forward',
 'slightly',
 'later',
 'long',
 'inside']

In [1021]:
# Top comparative adverbs
doc = sp(text_all)
doc = [i.text.lower() for i in doc if i.text not in STOP_WORDS and i.text not in string.punctuation and i.tag_=="RBR"]
# Or stopwords.words('english')
doc=pd.Series(doc)
top10_RBR=doc.value_counts()[:10].index.tolist() 
top10_RBR

['longer',
 'better',
 'sooner',
 'faster',
 'earlier',
 'closer',
 'later',
 'slower',
 'more',
 'quicker']

In [999]:
f = open('opinion-lexicon-English\\negative-words.txt','r')

neg_line = f.readline()
while neg_line.startswith(';'):   # Skip Header
    neg_line = f.readline()
neg_txt = f.read()
neg_txt = nltk.word_tokenize(neg_txt)
f.close()

f = open('opinion-lexicon-English\\positive-words.txt','r')

pos_line = f.readline()
while pos_line.startswith(';'):
    pos_line = f.readline()
pos_txt = f.read()
pos_txt = nltk.word_tokenize(pos_txt)
f.close()
pos_txt

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation',
 'accolade',
 'accolades',
 'accommodative',
 'accomodative',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accomplishments',
 'accurate',
 'accurately',
 'achievable',
 'achievement',
 'achievements',
 'achievible',
 'acumen',
 'adaptable',
 'adaptive',
 'adequate',
 'adjustable',
 'admirable',
 'admirably',
 'admiration',
 'admire',
 'admirer',
 'admiring',
 'admiringly',
 'adorable',
 'adore',
 'adored',
 'adorer',
 'adoring',
 'adoringly',
 'adroit',
 'adroitly',
 'adulate',
 'adulation',
 'adulatory',
 'advanced',
 'advantage',
 'advantageous',
 'advantageously',
 'advantages',
 'adventuresome',
 'adventurous',
 'advocate',
 'advocated',
 'advocates',
 'affability',
 'affable',
 'affably',
 'affectation',
 'affection',
 'affectionate',
 'affinity',
 'affirm',
 'affirmation',
 'affirmative',
 'affluence',
 'affluent',
 'afford',
 'affordable',
 'af

In [1002]:
'disappointment' in neg_txt

True

In [1054]:
# Try word2vec or normalize by individual word frequenceis.... also now there are three terms that should all be checked for being in punct stop etc
# Test
'disappointment' in neg_txt
'incredible' in neg_txt

summary = ''

for i in sent_tokenize(text_all):
    include = False
    type_i = []
    for j in word_tokenize(i):
        if j in neg_txt:
            include = True
            type_i = type_i + ['Negative sentiment']
        elif j in pos_txt:
            include = True
            type_i = type_i + ['Positive sentiment']
        if j in top_NNs:
            include = True
        if j in top10_RBR:
            include = True
        if j in top10_JJR: 
            include = True
    if include:
        summary = summary + i+'\n\n'

print(summary)

Bigger batteries for more electric range.

While in Brussels, Belgium to  drive the 2022 Volvo C40 Recharge  I was given a first look—and some first drive time—at the 2022 Volvo XC60 Recharge T8 plug-in hybrid, which gets that plus more power, and an updated infotainment system.

Volvo also anticipates a larger federal tax credit.

The 2022 Volvo XC60 plug-in hybrid arrives in the U.S. later this year or early in 2022.

Power, range, battery, and one-pedal driving
The largest change starts in the center tunnel of the 2022 XC60 where the battery’s been upgraded to a “three-layer pack,” as Volvo refers to it, with 18.7 kwh of capacity (of which 14.8 kwh is usable) up from 11.6 kwh (of which 9.1 kwh was usable).

A more powerful electric motor on the rear axle ups output from 87 hp to 142 hp with total system output from the turbocharged and supercharged inline-4 and PHEV system jumping from 400 hp and 472 lb-ft of torque to 455 hp and 523 lb-ft of torque.

While the gas engine isn’t more

In [1066]:
for w in top_NNs[0]:
    inds = re.findall(w, text_all)
    for i in inds:
        print(text_all[i-20:i+20])
        #print(text_all[np.max(0,i-20):np.min(len(toks)-1,i+20)])

          
#toks = word_tokenize(text_all)
#for i in range(0,len(toks)):
#    if toks[i] in top_NNs:
#        print(toks[i])
#        context = toks[np.max(0,i-5):np.min(len(toks)-1,i+5)]
#        print(context)


TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [1076]:
inds = re.finditer('gas engine', text_all.lower())

for match in inds:
    s = match.start()
    e = match.end()
    print(s,e)

1229 1239
2851 2861
3552 3562
61361 61371
90980 90990
164714 164724
165952 165962
166139 166149
171628 171638
172625 172635
172927 172937
173575 173585
174084 174094
174181 174191


In [1087]:
# Initialize
matcher = PhraseMatcher(sp.vocab)

#create the list of words to match
word_list = top_2g_filt
#obtain doc object for each word in the list and store it in a list
patterns = [sp(w) for w in word_list]
#add the pattern to the matcher
matcher.add("PATTERNS", patterns)
text_all = text_all.replace("\n", " ")
doc = sp(text_all)
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start-5:end+5]
    print(span.text)
    



Volvo also anticipates a larger federal tax credit. The 2022 Volvo
range, battery, and one-pedal driving The largest change starts in
of torque. While the gas engine isn’t more powerful,
for 2022. The larger battery pack and all-electric range
increase the XC60’s eligible federal tax credit status from the current
a fully electric vehicle: one-pedal driving capability thanks to regenerative braking
pedal driving capability thanks to regenerative braking. For 2022 the XC60
about the upgraded PHEV: one-pedal driving has indeed arrived in Volvo
powerful electric motor and larger battery pack, if the accelerator pedal
Pure drive mode, the gas engine will kick in to provide
-pedal feature with more regenerative braking works, but the deceleration
the speed the stronger the regenerative braking. It wasn’t possible
at low speeds when the gas engine is running, and perhaps
Volvo to get the upgraded battery pack, motors, and infotainment
the nicely weighted steering that doesn’t feel overbooste

In [725]:
set1 = {'Negative sentiment','Hi'}
set1.update({'Negative sentiment'})
set1

{'Hi', 'Negative sentiment'}

In [ ]:
displacy.render(text1,style="ent",jupyter=True)